In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("data/Netflix_Data.csv", index_col=0)
df = data.copy()
df.head()

In [ ]:
df["Day"] = pd.to_datetime(df["Date"]).dt.day
df["Month"] = pd.to_datetime(df["Date"]).dt.month
df["Year"] = pd.to_datetime(df["Date"]).dt.year
df.drop(columns=["Date"], inplace=True)

df["Next_Close"] = df["Close"].shift(-1)

df = df.dropna()
df.head()

In [ ]:
X = df.drop(columns=["Next_Close"], axis=1)
Y = df["Next_Close"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
model = SVR(kernel='rbf')
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2],
    'gamma': ['scale', 'auto', 0.1, 1]
}

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(x_train, y_train)

In [ ]:
model_tuned = grid_search.best_estimator_
y_pred_tuned = model_tuned.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred_tuned))

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(y_test.values, label='Actual', color='blue')
plt.plot(y_pred_tuned, label='Predicted', color='red')
plt.xlabel("Time Index")
plt.ylabel("Stock Price (USD)")
plt.title('Netflix Stock Price Prediction')
plt.legend()
plt.show()